In [62]:
import pandas as pd
import matplotlib.pyplot as plt
import random
import numpy as np
import keras
import tensorflow
from keras import Sequential
from keras.layers import Dense
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics, svm
from sklearn.metrics import average_precision_score

In [44]:
df = pd.read_csv('COVID19_open_line_list.csv')

In [45]:
#df = df[pd.notnull(df['age'])]
#df = df[pd.notnull(df['sex'])]
#df = df[pd.notnull(df['country'])]
df = df[pd.notnull(df['symptoms'])]
#df = df[pd.notnull(df['lives_in_Wuhan'])]
df.drop(columns = ['city','province','latitude','longitude','geo_resolution','additional_information','source','sequence_available','outcome','notes_for_discussion','location','admin3','admin2','admin1','country_new','admin_id','data_moderator_initials'])

,ID,age,sex,country,wuhan(0)_not_wuhan(1),date_onset_symptoms,date_admission_hospital,date_confirmation,symptoms,lives_in_Wuhan,travel_history_dates,travel_history_location,reported_market_exposure,chronic_disease_binary,chronic_disease,date_death_or_discharge
30,6,NaN,NaN,China,1,NaN,NaN,24.01.2020,pneumonia,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31,11,NaN,NaN,China,1,NaN,NaN,23.01.2020,pneumonia,NaN,NaN,NaN,NaN,NaN,NaN,NaN
243,10913,80-89,male,Japan,1,09.02.2020,NaN,18.02.2020,"dyspnea, fever",NaN,NaN,NaN,NaN,NaN,NaN,NaN
244,11272,80-89,female,Japan,1,22.01.2020,01.02.2020,13.02.2020,"anorexia, fatigue",NaN,NaN,NaN,NaN,NaN,NaN,13.02.2020
245,11553,80-89,male,Japan,1,27.01.2020,19.02.2020,21.02.2020,anorexia,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12372,12552,NaN,NaN,Japan,1,16.02.2020,25.02.2020,26.02.2020,cough,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12664,12892,NaN,NaN,Nigeria,1,NaN,NaN,27.02.2020,no serious symptoms,NaN,25.02.2020,"Milan, Italy",NaN,NaN,NaN,NaN
13186,13436,NaN,female,Sweden,1,NaN,NaN,27.02.2020,similar to a respiratory infection,no,NaN,Germany,NaN,NaN,NaN,NaN
13197,13478,NaN,female,Netherlands,1,NaN,NaN,28.02.2020,mild,NaN,NaN,returned from Italy this week,NaN,NaN,NaN,NaN


In [46]:
test = df.iloc[:,13]
symptom_set = set()
for row in list(test):
    row = str(row)
    l = row.split(",")
    for symp in l:
        symp = symp.strip()
        if symp == "nan":
            continue
        elif symp not in symptom_set:
            if "fever" in symp or "Fever" in symp:
                symptom_set.add("fever")
            else:
                symptom_set.add(symp)
        else:
            continue
print(symptom_set)

{'no serious symptoms', 'anhelation', 'sore limbs', 'no symptoms', 'sweating', 'expectoration', 'chest pain', 'severe dyspnea', 'similar to a respiratory infection', 'respiratory symptoms', 'headache.', 'shortness breath', 'no respiratory symptoms', 'chest tightness', 'conjunctivitis', 'acute pharyngitis', 'wheezing', 'Sore throat', 'dry throat', 'lack of energy', 'pharyngeal discomfort', 'rhinorrhea', 'cough; sore body; cold', 'sputum', 'discomfort', 'flu-like symptoms', 'muscle aches', 'rhinorrhoea', 'Pharyngeal dryness', 'myalgias', 'mild', 'diarrheoa', 'weakness', 'muscle soreness', 'diarrhoea', 'pneumonia', 'systemic weakness', 'feeling ill', 'Discharge', 'breathing difficulty', 'acute respiratory viral infection', 'mild coughing', 'Sputum', 'rever', 'Headache', 'muscular soreness', 'asymptomatic', 'fever', 'couh', 'toothache', 'then 38 ℃)', 'other symptoms', 'Fatigue', 'headache; dry mouth', 'fatigue', 'aching muscles', 'muscle pain', 'difficulty walking', 'difficulty breathing',

In [47]:
test = df.iloc[:,5]
country_dict = dict()
count = 0
for row in list(test):
    row = str(row)
    if row in country_dict:
        continue
    else:
        country_dict[row] = count
        count+= 1
print(country_dict)

{'China': 0, 'Japan': 1, 'nan': 2, 'United States': 3, 'Thailand': 4, 'Vietnam': 5, 'South Korea': 6, 'Singapore': 7, 'Malaysia': 8, 'Spain': 9, 'Cambodia': 10, 'Shaanxi': 11, 'France': 12, 'Lebanon': 13, 'Philippines': 14, 'Germany': 15, 'Lithuania': 16, 'Nepal': 17, 'Italy': 18, 'Russia': 19, 'Belgium': 20, 'Nigeria': 21, 'Sweden': 22, 'Netherlands': 23, 'Ecuador': 24}


In [48]:
cleanedData = []
count = 0
Count = 0
for index,row in df.iterrows():
    cur = []
    Count += 1
    '''
    # Clean the age column
    if str(row["age"]).isnumeric():
        #cur.append(int(row["age"]) // 10)
        cur += [0] * (int(row["age"]) // 10) + [1] + (9 - (int(row["age"]) // 10)) * [0]
    else:
        if "-" in row["age"]:
            loc = row["age"].index("-")
            #cur.append(int(row["age"][:loc]) // 10)
            cur += [0] * (int(row["age"][:loc]) // 10) + [1] + (9 - (int(row["age"][:loc]) // 10)) * [0]
        else:
            #cur.append(int(float(row["age"]) // 10))
            cur += [0] * ((int(float(row["age"]))) // 10) + [1] + (9 - ((int(float(row["age"]))) // 10)) * [0]
            
    '''
    # Clean the sex column
    if row["sex"] == "male":
        cur += [1,0]
    else:
        cur += [0,1]
    '''
    # Clean the country
    #cur.append(country_dict[str(row["country"])])
    cur += [0]*(country_dict[str(row["country"])]) + [1] + (26 -country_dict[str(row["country"])]) * [0]
    
    # Clean the Wuhan column, check whether they are citizens in Wuhan, 1 means not Wuhan
    cur.append(row["wuhan(0)_not_wuhan(1)"])
    # Clean the sympton date
    '''
    # Clean the sympton
    temp = str(row["symptoms"]).lower()
    # fever, cough, pneumonitis, fatigue
    # Fever- related
    if "fever" in temp or "headache" in temp:
        cur.append(1)
    else:
        cur.append(0)
    # Cough related
    if "cou" in temp or "throa" in temp or "dry" in temp or "pharyngeal" in temp or "expectoration" in temp or "flu" in temp:
        cur.append(1)
    else:
        cur.append(0)
    # Cold related
    if "chill" in temp or "nose" in temp or "nasal" in temp or "sneez" in temp:
        cur.append(1)
    else:
        cur.append(0)
    # pneumonitis related
    if "pneumon" in temp or "respiratory" in temp or "breath" in temp:
        cur.append(1)
    else:
        cur.append(0)
    # Muscle or fatigue, physical issue related
    if "fatigue" in temp or "myalgias" in temp or "musc" in temp or "walk" in temp or "chest" in temp or "limbs" in temp or "joint" in temp or "physical" in temp or "energy" in temp:
        cur.append(1)
    else:
        cur.append(0)
    # Stomacha related
    if "diarrhoea" in temp or "abdominal" in temp:
        cur.append(1)
    else:
        cur.append(0)
    # Other symptom or non-symptom
    if len(temp) > 0 and temp != "nan":
        cur.append(1)
    else:
        cur.append(0)
    
    # Clean the column whether live in Wuhan or have travel relations with Wuhan
    if str(row["lives_in_Wuhan"]).lower() == "nan" or str(row["lives_in_Wuhan"]).lower() == "no" or "wuhan" not in str(row["travel_history_location"]).lower():
        cur.append(0)
    else:
        cur.append(1)
        
    # Clean the target value: date_death_or_discharge
    if str(row["date_death_or_discharge"]).lower() == "nan":
        count += 1
        cur.append(0)
    else:
        cur.append(1)
    cleanedData.append(cur)
    
#print(count,Count)
#print(cleanedData)

In [49]:
print(cleanedData)

[[0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0], [0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0], [1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0], [0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1], [1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], [1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0], [1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0], [1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0], [1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0], [0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0], [1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0], [1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0], [0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0], [1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0], [1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0], [1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0], [1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0], [0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0], [0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0], [1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0], [1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0], [1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0], [1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0], [0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0], [1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0], [1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0], [1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0], [1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0], [1, 0, 1, 0, 0, 0, 

In [50]:
import csv
f = open("NewCleanedConfirmedCase.csv", "a")
fieldnames = ['male','female','Fever','Cough','Cold','Pneumonitis','Fatigue','Stomacha','Other Symptoms','Relation with Wuhan','Death or Not']
writer = csv.DictWriter(f, fieldnames=fieldnames)
writer.writeheader()
writer = csv.writer(f)
writer.writerows(cleanedData)
f.close()

In [52]:
df = pd.read_csv("NewCleanedConfirmedCase.csv")

In [53]:
df

,male,female,Fever,Cough,Cold,Pneumonitis,Fatigue,Stomacha,Other Symptoms,Relation with Wuhan,Death or Not
0,0,1,0,0,0,1,0,0,1,0,0
1,0,1,0,0,0,1,0,0,1,0,0
2,1,0,1,0,0,0,0,0,1,0,0
3,0,1,0,0,0,0,1,0,1,0,1
4,1,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
489,0,1,0,1,0,0,0,0,1,0,0
490,0,1,0,0,0,0,0,0,1,0,0
491,0,1,0,0,0,1,0,0,1,0,0
492,0,1,0,0,0,0,0,0,1,0,0


In [57]:
X = df.iloc[:,:-1]
Y = df.iloc[:,-1]
print(X)
print(Y)

     male  female  Fever  Cough  Cold  Pneumonitis  Fatigue  Stomacha  \
0       0       1      0      0     0            1        0         0   
1       0       1      0      0     0            1        0         0   
2       1       0      1      0     0            0        0         0   
3       0       1      0      0     0            0        1         0   
4       1       0      0      0     0            0        0         0   
..    ...     ...    ...    ...   ...          ...      ...       ...   
489     0       1      0      1     0            0        0         0   
490     0       1      0      0     0            0        0         0   
491     0       1      0      0     0            1        0         0   
492     0       1      0      0     0            0        0         0   
493     0       1      1      0     0            0        0         0   

     Other Symptoms  Relation with Wuhan  
0                 1                    0  
1                 1                  

In [96]:
from sklearn.metrics import precision_recall_fscore_support
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=1) 
# 80% training and 20% test
clf = DecisionTreeClassifier()
clf = clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
#print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
# precision, recall ,f1, confusion matrix
precision_recall_fscore_support(y_test, y_pred, average='weighted')

(0.9501197542434656, 0.9595959595959596, 0.9530723905723905, None)

In [75]:
import xgboost as xgb
